In [ ]:
"""_summary_

Variations to aggregate over: 
    + per trip
    + trip fuel increased by the end of the trip?
    
    """

'_summary_\n\nVariations to aggregate over: \n    + per day as challenge 1 \n    + using consecutive trip ids\n\n    '

In [3]:
import pandas as pd
import pyarrow.parquet as pa
from pathlib import Path
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [4]:
data_path = '/Users/athulparvelikudy/Personal/ACRTA/tech-test-data/supporting-data'
file_name_0 = 'drive/part-00000-tid-4109877695252048813-a3139a95-1807-419c-af03-4877385b4c8c-11-1-c000.snappy.parquet'
file_name_1 = 'drive/part-00001-tid-4109877695252048813-a3139a95-1807-419c-af03-4877385b4c8c-12-1-c000.snappy.parquet'
file_name_2 = 'drive/part-00002-tid-4109877695252048813-a3139a95-1807-419c-af03-4877385b4c8c-13-1-c000.snappy.parquet'
file_name_3 = 'drive/part-00003-tid-4109877695252048813-a3139a95-1807-419c-af03-4877385b4c8c-14-1-c000.snappy.parquet'
file_name_4 = 'drive/part-00004-tid-4109877695252048813-a3139a95-1807-419c-af03-4877385b4c8c-15-1-c000.snappy.parquet'
vehicle_file = 'vehicle.csv'

vehicle_csv = str(Path(data_path) / vehicle_file)

full_file_path0 = str(Path(data_path) / file_name_0)
full_file_path1 = str(Path(data_path) / file_name_1)
full_file_path2 = str(Path(data_path) / file_name_2)
full_file_path3 = str(Path(data_path) / file_name_3)
full_file_path4 = str(Path(data_path) / file_name_4)



In [5]:
spark = (
    SparkSession.builder
    .appName("ReadLocalParquet")
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/27 21:26:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/27 21:26:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/12/27 21:26:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [6]:
df_0 = spark.read.parquet(full_file_path0)
df_vehicle = (
    spark.read
    .option("header", "true") 
    .option("inferSchema", "true") 
    .csv(vehicle_csv)
)

In [7]:
df_0.createOrReplaceTempView("drive_data")
df_vehicle.createOrReplaceTempView("vehicle_specs")


In [8]:
drive_w_specs = spark.sql(
    """
    SELECT
        d.trip_id,
        d.datetime,
        d.vehicle_spec_id,
        d.eng_load,
        d.velocity,
        d.fuel_level,
        v.fuel_tank_capacity
    FROM drive_data d
    INNER JOIN vehicle_specs v
        ON d.vehicle_spec_id = v.vehicle_spec_id
""")

In [9]:
drive_w_specs.show(10, truncate=False)

drive_w_specs.createOrReplaceTempView("drive_w_specs")

+--------------------------------+-------------------+---------------+--------+--------+----------+------------------+
|trip_id                         |datetime           |vehicle_spec_id|eng_load|velocity|fuel_level|fuel_tank_capacity|
+--------------------------------+-------------------+---------------+--------+--------+----------+------------------+
|1e20465533c545f98332ff14d5a0af22|2017-01-06 14:41:24|1000502        |210.65  |61.01   |76.0      |55                |
|afe9ae193f1544398c289a0cb3ff611e|2017-02-14 08:21:22|1000501        |210.18  |70.2    |75.0      |60                |
|2df4799783604dd9bcf6c2692a6a6bf2|2017-02-14 07:08:26|1000502        |197.89  |68.41   |59.0      |55                |
|96c2aec9925445acb430607c2cd944cf|2017-01-08 11:19:41|1000503        |207.9   |69.44   |96.68     |66                |
|472c62cc56b8421f8c8080a2d2f14f99|2017-01-06 21:30:14|1000500        |214.13  |78.5    |198.19    |47                |
|af2509615c5c4b2db121d08dfba01f34|2017-01-06 05:

In [10]:
trip_agg = spark.sql(
    """

       SELECT
        trip_id,
        AVG(100.0 * (eng_load / 255.0)) AS average_eng_load_perc,
        AVG(velocity) AS average_velocity
    FROM drive_w_specs
    GROUP BY trip_id

    """ 
)
trip_agg.show(10, truncate=False)

trip_agg.createOrReplaceTempView("trip_agg")

+--------------------------------+---------------------+------------------+
|trip_id                         |average_eng_load_perc|average_velocity  |
+--------------------------------+---------------------+------------------+
|b51ed58c5c4b4081a44b4a839b9e664e|75.38118335053318    |70.43953947368419 |
|6777f0df4b9f4053a129a8ea904fc3d6|76.06240715316648    |71.95276481149018 |
|d149fb184c7f40e193d346ed70758971|84.6334848288102     |67.94728476821192 |
|8c8e8c3ade4b4e77bfe5425835d756ac|85.9261958997722     |53.42166287015945 |
|06e692ceefb94c29b4b8534b5efd8109|83.1657560095168     |51.13874476987442 |
|f5e8ac7448174aeea872fc73a361f1c6|78.01501811224749    |73.81612679544325 |
|cb6d229addab403ab597dd3c460baee8|85.16109403110832    |63.66318234610912 |
|06d25867aaba41978ca77da69f404793|81.18103479753175    |64.10868404717574 |
|eeb1e4abfb5f4005a6d82b367e37040c|75.6065374536911     |58.328191244239655|
|c3ffb058fe0b40eea84a771b42305a78|75.29775429849039    |49.7653733602421  |
+-----------

In [11]:
ranked = spark.sql(
    """

       SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY trip_id ORDER BY datetime ASC)  AS rn_start,
        ROW_NUMBER() OVER (PARTITION BY trip_id ORDER BY datetime DESC) AS rn_end
    FROM drive_w_specs

    """ 
)
ranked.show(10, truncate=False)

+--------------------------------+-------------------+---------------+--------+--------+----------+------------------+--------+------+
|trip_id                         |datetime           |vehicle_spec_id|eng_load|velocity|fuel_level|fuel_tank_capacity|rn_start|rn_end|
+--------------------------------+-------------------+---------------+--------+--------+----------+------------------+--------+------+
|05e3f9ef5204475981edd44e5939c4a4|2017-02-01 17:04:37|1000504        |222.77  |20.0    |116.0     |66                |779     |1     |
|05e3f9ef5204475981edd44e5939c4a4|2017-02-01 17:04:32|1000504        |214.75  |55.68   |124.0     |66                |778     |2     |
|05e3f9ef5204475981edd44e5939c4a4|2017-02-01 17:04:31|1000504        |206.32  |69.41   |115.0     |66                |777     |3     |
|05e3f9ef5204475981edd44e5939c4a4|2017-02-01 17:04:26|1000504        |208.72  |70.1    |119.0     |66                |776     |4     |
|05e3f9ef5204475981edd44e5939c4a4|2017-02-01 17:04:13|1

In [14]:
ranked.show(10, truncate=False)

ranked.createOrReplaceTempView("ranked")

+--------------------------------+-------------------+---------------+--------+--------+----------+------------------+--------+------+
|trip_id                         |datetime           |vehicle_spec_id|eng_load|velocity|fuel_level|fuel_tank_capacity|rn_start|rn_end|
+--------------------------------+-------------------+---------------+--------+--------+----------+------------------+--------+------+
|05e3f9ef5204475981edd44e5939c4a4|2017-02-01 17:04:37|1000504        |222.77  |20.0    |116.0     |66                |779     |1     |
|05e3f9ef5204475981edd44e5939c4a4|2017-02-01 17:04:32|1000504        |214.75  |55.68   |124.0     |66                |778     |2     |
|05e3f9ef5204475981edd44e5939c4a4|2017-02-01 17:04:31|1000504        |206.32  |69.41   |115.0     |66                |777     |3     |
|05e3f9ef5204475981edd44e5939c4a4|2017-02-01 17:04:26|1000504        |208.72  |70.1    |119.0     |66                |776     |4     |
|05e3f9ef5204475981edd44e5939c4a4|2017-02-01 17:04:13|1

In [15]:
fuel = spark.sql(
    """

       SELECT
        trip_id,

        -- fuel at trip start
        MAX(
            CASE WHEN rn_start = 1
            THEN (fuel_level / 255.0) * fuel_tank_capacity
            END
        ) AS start_fuel_litres,

        -- fuel at trip end
        MAX(
            CASE WHEN rn_end = 1
            THEN (fuel_level / 255.0) * fuel_tank_capacity
            END
        ) AS end_fuel_litres
    FROM ranked
    GROUP BY trip_id

    """ 
)
fuel.show(10, truncate=False)

fuel.createOrReplaceTempView("fuel")    


+--------------------------------+------------------+------------------+
|trip_id                         |start_fuel_litres |end_fuel_litres   |
+--------------------------------+------------------+------------------+
|05e3f9ef5204475981edd44e5939c4a4|29.764705882352942|30.023529411764706|
|06d25867aaba41978ca77da69f404793|23.058823529411764|20.705882352941178|
|06e692ceefb94c29b4b8534b5efd8109|15.764705882352942|17.647058823529413|
|1096a786b76140909b47a8fd25b9b0d8|42.98541176470589 |46.88070588235294 |
|137610fca6804dbc8ac9576bdf774238|28.589215686274514|29.687058823529412|
|1ca7d052a83b454691224c4660b712af|22.823529411764703|21.88235294117647 |
|1ffd8282f0d74ce9843f66ad57dc4c9a|17.901960784313726|18.333333333333332|
|2346d694cdf54defb22be7743dec8c7c|32.446745098039216|31.63356862745098 |
|25bee519a37649b197e5505aff719991|31.722352941176467|32.88235294117647 |
|25c87a2432e4403598b57a863216e87c|35.58607843137255 |35.78666666666666 |
+--------------------------------+-----------------

In [18]:
final = spark.sql(
    """

       SELECT
        a.trip_id,
        a.average_eng_load_perc,
        a.average_velocity,
        COALESCE(f.start_fuel_litres, 0.0)
        - COALESCE(f.end_fuel_litres, 0.0)
        AS fuel_used
    FROM trip_agg a
    INNER JOIN fuel f
        ON a.trip_id = f.trip_id
    ORDER BY trip_id

    """ 
)
final.show(10, truncate=False)



+--------------------------------+---------------------+------------------+--------------------+
|trip_id                         |average_eng_load_perc|average_velocity  |fuel_used           |
+--------------------------------+---------------------+------------------+--------------------+
|00922df3be5a4589ab385d0c2da2dd81|79.84385982198573    |56.51061976549414 |-0.39654901960783917|
|01b8a24510cd4e4684d67b96369286e0|75.42454611474217    |65.14074074074077 |-3.185294117647061  |
|0244f7bc747b41fba6fcd75444736621|84.27522664979973    |65.51698924731178 |-0.0329411764705938 |
|02c51e56cc484711b218d3d01196687a|79.18065029438276    |54.334968440036036|0.9210980392156856  |
|039c14a9a7df4096bb6e9445136fdf77|80.47147746079835    |65.90414878397712 |-0.8627450980392126 |
|03b8ac1525474754b4d89d2d647aa8e4|78.86301627033791    |60.15420803782501 |4.266666666666666   |
|047633791fb243ff814b1b760a7bd5a4|81.30557099221605    |70.73088442211053 |-2.8310588235294105 |
|04a0c61b0db5478e8e8c6052b300a